In [ ]:
# !pip install poe-api
!pip install -U poe-api-wrapper

In [2]:
from poe_api_wrapper import PoeApi
# client = PoeApi("3YCojFePwaecuxQp1n391g%3D%3D")
client = PoeApi("51uUHatfZv-V8_dFmESKMg%3D%3D")
# client = PoeApi("p49wzQYujpnuLVWJ8eOKDw%3D%3D")

# Using Client with proxy (default is False)
# client = PoeApi("TOKEN_HERE", proxy=True)

**Mounted drive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**get data**

In [4]:
import pandas as pd

In [6]:
df = pd.read_csv("/content/drive/MyDrive/largeProject/code/QA/MRC.csv",index_col=False)

In [7]:
def get_prompt(context, question):
    prompt = f'''Ngữ cảnh: "{context}". Câu hỏi "{question}". Trả lời câu hỏi trên theo từng bước dựa vào thông tin trong ngữ cảnh với tối đa 100 từ.'''
    return prompt

In [8]:
# print(client.get_chat_history('chinchilla'))
dict_client = client.get_chat_history()

In [9]:
list_chat_code = []

for dict_id in dict_client['acouchy']:
  list_chat_code.append(dict_id['chatCode'])

In [10]:
len(list_chat_code)

18

In [ ]:
dict_client['acouchy']

In [11]:
df['prompt'] = df.apply(lambda x: get_prompt(x['context'], x['question']), axis=1)

In [12]:
df['prompt'].to_list()[:10]

['Ngữ cảnh: "Dữ liệu cho thấy trong q2 năm 2023, chỉ tiêu Tỷ lệ hài lòng của KH dịch vụ chuyển phát của tổng công ty VTPOST được đánh gía là không đạt chỉ tiêu hiện đã chỉ bằng 0.88 lần so với mục tiêu kpi ban đầu đề ra là 97.2%. Đối với kpi quý trước có được là 95.9%. Giảm 3.8% so với kết quả là với 2/2022". Câu hỏi "kpi quý trước là bao nhiêu?". Trả lời câu hỏi trên theo từng bước dựa vào thông tin trong ngữ cảnh với tối đa 100 từ.',
 'Ngữ cảnh: "Dữ liệu cho thấy trong q2 năm 2023, chỉ tiêu Tỷ lệ hài lòng của KH dịch vụ chuyển phát của tổng công ty VTPOST được đánh gía là không đạt chỉ tiêu hiện đã chỉ bằng 0.88 lần so với mục tiêu kpi ban đầu đề ra là 97.2%. Đối với kpi quý trước có được là 95.9%. Giảm 3.8% so với kết quả là với 2/2022". Câu hỏi "tỷ lệ hài lòng của KH dịch vụ chuyển phát bằng bao nhiêu lần so với kpi đề ra". Trả lời câu hỏi trên theo từng bước dựa vào thông tin trong ngữ cảnh với tối đa 100 từ.',
 'Ngữ cảnh: "Dữ liệu cho thấy trong q2 năm 2023, chỉ tiê

In [13]:
import os
import random
from tqdm import tqdm
import time

In [14]:
list_res = []
list_prompt = []
list_index = []

for index, row in tqdm(df.iterrows()):
    if index <= 3740:
      continue
    prompt = row['prompt']
    res = ""
    while True:
        try:
            for chunk in client.send_message("acouchy", prompt, chatCode = list_chat_code[index % len(list_chat_code)], timeout=30):
                pass
            res = chunk["text"]
            if res == "You are sending and receiving too many words in a short period of time.":
                continue
            break
        except Exception as e:
            print(e)
            if "Timed out waiting" in str(e):
                client = PoeApi("51uUHatfZv-V8_dFmESKMg%3D%3D")
                dict_chat = client.get_chat_history()["acouchy"]
                list_chat_code = [each['chatCode'] for each in dict_chat]
                time.sleep(10)
            res = ""
    res = " ".join(res.split("\n")).strip()

    list_res.append(res)
    list_prompt.append(prompt)
    list_index.append(index)

    if index % 50 == 0 and index != 0:
        time.sleep(10)
    if index % 100 == 0:
      dict_list = {
        "index" : list_index,
        "prompt" : list_prompt,
        "res" : list_res
      }

      if 'mrc_history2.csv' not in os.listdir('/content/drive/MyDrive/largeProject'):
          df_test = pd.DataFrame.from_dict(dict_list)
          df_test.to_csv('/content/drive/MyDrive/largeProject/mrc_history2.csv', index=False)
      else:
          df_test = pd.read_csv('/content/drive/MyDrive/largeProject/mrc_history2.csv',index_col=None)
          df_test = pd.concat([df_test, pd.DataFrame.from_dict(dict_list)], ignore_index=True)
          df_test.to_csv('/content/drive/MyDrive/largeProject/mrc_history2.csv', index=False)
    if index % 200 == 0 and index != 0:
      try:
        for idx_chat in list_chat_code:
            amount_previous_messages = len(client.get_previous_messages("acouchy", chatCode=idx_chat, get_all=True))
            if amount_previous_messages < 50:
                continue
            client.purge_conversation("acouchy", chatCode=idx_chat, count=30)
        time.sleep(20)
      except Exception as e:
          print(e)

2810it [00:19, 28091.59it/s]


Retrying request 1/3 times...


3745it [01:20, 41.59it/s]


Retrying request 1/3 times...


3746it [01:47, 23.36it/s]

...


Retrying request 1/3 times...


4112it [3:44:11, 30.61s/it]


Retrying request 1/3 times...


4112it [3:44:45,  3.28s/it]


KeyboardInterrupt: ignored

In [ ]:
dict_list = {
  "index" : list_index,
  "prompt" : list_prompt,
  "res" : list_res
}

df_test = pd.read_csv('/content/drive/MyDrive/largeProject/mrc_history2.csv',index_col=None)
df_test = pd.concat([df_test, pd.DataFrame.from_dict(dict_list)], ignore_index=True)
df_test.to_csv('/content/drive/MyDrive/largeProject/mrc_history2.csv', index=False)

In [ ]:
list_res

[(0, 'Theo ngữ cảnh, KPI quý trước là 95,9%.'),
 (1,
  'Tỷ lệ hài lòng của KH dịch vụ chuyển phát chỉ bằng 0,88 lần so với KPI đề ra.'),
 (2, 'Mục tiêu đề ra là 97,2%.')]

In [ ]:
text = "chỉ tiêu Tỷ lệ hài lòng của KH dịch vụ chuyển phát của tổng công ty VTPOST được đánh gía là không đạt chỉ tiêu hiện đã chỉ bằng 0.88 lần so với mục tiêu kpi ban đầu đề ra là 97.2%. Đối với kpi quý trước có được là 95.9%. Giảm 3.8% so với kết quả là với 2/2022. Câu hỏi kpi quý trước là bao nhiêu?"

print(text, end="", flush=True)

chỉ tiêu Tỷ lệ hài lòng của KH dịch vụ chuyển phát của tổng công ty VTPOST được đánh gía là không đạt chỉ tiêu hiện đã chỉ bằng 0.88 lần so với mục tiêu kpi ban đầu đề ra là 97.2%. Đối với kpi quý trước có được là 95.9%. Giảm 3.8% so với kết quả là với 2/2022. Câu hỏi kpi quý trước là bao nhiêu?